In [ ]:
%load_ext autoreload
%autoreload 2

import nibabel as nib
import SimpleITK as sitk
import matplotlib.pyplot as plt

import codebase.codebase_settings as cbs
from codebase.preprocessor.images import beams_eye_view as bev

In [ ]:
def show_image(img, location):
    print(img.GetDirection())
    print('size: ' + str(img.GetSize()))
    print('spacing: ' + str(img.GetSpacing()))
    print('origin: ' + str(img.GetOrigin()))
    print('corner 1: ' + str(img.TransformIndexToPhysicalPoint([0, 0, 0])))
    orient = sitk.DICOMOrientImageFilter_GetOrientationFromDirectionCosines(img.GetDirection())
    print(f'Orientation: {orient}')
    nda = sitk.GetArrayFromImage(img)
    print(f'Array shape: {nda.shape}')
    fig, axes = plt.subplots(1, 3, figsize=(8, 24))
    axes[0].imshow(nda[location[0], :, :])
    axes[1].imshow(nda[:, location[1], :])
    axes[2].imshow(nda[:, :, location[2]])

In [ ]:
_TEST_DATA_FOLDER = cbs.CODEBASE_PATH / 'preprocessor' / 'images' / 'test_data' / 'images'
_TEST_IMAGE_FILE = _TEST_DATA_FOLDER / 'MDA-103__CT.nii.gz'

In [ ]:
bev_op = bev.BeamsEyeView()

In [ ]:
test_image = bev_op.read_image(_TEST_IMAGE_FILE)
show_image(test_image, location=(150, 200, 150))

In [ ]:
resampled_img = bev_op.resample(test_image, [2, 2, 2])

In [ ]:
show_image(resampled_img, location=(260, 120, 150))

In [ ]:
fake_iso = resampled_img.TransformIndexToPhysicalPoint([100, 150, 250])
aligned_img = bev_op.align_isocenter(resampled_img, isocenter=fake_iso)

In [ ]:
show_image(aligned_img, location=(260, 120, 150))

In [ ]:
reorient_img = bev_op.align_patient_position(aligned_img)

In [ ]:
show_image(reorient_img, location=(120, 250, 130))

In [ ]:
couch_kick_img = bev_op.couch_kick(reorient_img, 270)

In [ ]:
show_image(couch_kick_img, location=(140, 210, 130))

In [ ]:
gantry_rotate_img = bev_op.gantry_rotate(couch_kick_img, 40)

In [ ]:
show_image(gantry_rotate_img, location=(140, 210, 130))